In [1]:
import csv, copy
from collections import defaultdict
import numpy, re, math
from sklearn.linear_model import Ridge, LinearRegression
from string import punctuation
from scipy import stats
import pandas
from collections import defaultdict
from sklearn import linear_model, datasets
from collections import Counter
import random

In [3]:
data = pandas.read_csv("h1b_kaggle.csv")
data = data.set_index('Unnamed: 0')
# Removing rows with NA in case_status
data = data[pandas.notnull(data['CASE_STATUS'])]
data = data.drop(data[data.CASE_STATUS == 'REJECTED'].index)
data = data.drop(data[data.CASE_STATUS == 'INVALIDATED'].index)
data = data.drop(data[data.CASE_STATUS == 'WITHDRAWN'].index)
data = data.drop(data[data.CASE_STATUS == 'PENDING QUALITY AND COMPLIANCE REVIEW - UNASSIGNED'].index)

# To keep rows which have employer names that are among the ~1200 most frequent
threshold = 0.0001
#for col in df:
col = 'EMPLOYER_NAME'
counts = data[col].value_counts(normalize=True)
data = data.loc[data[col].isin(counts[counts > threshold].index), :]

# Case_status = 1 for certified/certified-withdrawn, 0 for denied
data['CASE_STATUS'] = data['CASE_STATUS'].map({'CERTIFIED': 1, 'DENIED': 0, 'CERTIFIED-WITHDRAWN': 2})
data['CASE_STATUS'].replace(2,1, inplace=True)
data['FULL_TIME_POSITION'] = data['FULL_TIME_POSITION'].map({'Y': 1, 'N': 0})

# Separating states and adding that as another column
states = []
for row in data.iterrows():
    st = row[1]['WORKSITE'].split(',')[-1][1:]
    states.append(st)
s = pandas.Series(states)
data['STATES'] = s.values

In [4]:
state_map = {
    'ALABAMA': 1,
    'ALASKA': 2,
    'ARIZONA': 3,
    'ARKANSAS': 4,
    'CALIFORNIA': 5,
    'COLORADO': 6,
    'CONNECTICUT': 7,
    'DELAWARE': 8,
    'DISTRICT OF COLUMBIA': 9,
    'FLORIDA': 10,
    'GEORGIA': 11,
    'HAWAII': 12,
    'IDAHO': 13,
    'ILLINOIS': 14,
    'INDIANA': 15,
    'IOWA': 16,
    'KANSAS': 17,
    'KENTUCKY': 18,
    'LOUISIANA': 19,
    'MAINE': 20,
    'MARYLAND': 21,
    'MASSACHUSETTS': 22,
    'MICHIGAN': 23,
    'MINNESOTA': 24,
    'MISSISSIPPI': 25,
    'MISSOURI': 26,
    'MONTANA': 27,
    'NEBRASKA': 28,
    'NEVADA': 29,
    'NEW HAMPSHIRE': 30,
    'NEW JERSEY': 31,
    'NEW MEXICO': 32,
    'NEW YORK': 33,
    'NORTH CAROLINA': 34,
    'NORTH DAKOTA': 35,
    'OHIO': 36,
    'OKLAHOMA': 37,
    'OREGON': 38,
    'PENNSYLVANIA': 39,
    'PUERTO RICO': 40,
    'RHODE ISLAND': 41,
    'SOUTH CAROLINA': 42,
    'SOUTH DAKOTA': 43,
    'TENNESSEE': 44,
    'TEXAS': 45,
    'UTAH': 46,
    'VERMONT': 47,
    'VIRGINIA': 48,
    'WASHINGTON': 49,
    'WEST VIRGINIA': 50,
    'WISCONSIN': 51,
    'WYOMING': 52}

In [7]:
def feature(datum):
    feat = [0]*(1+6+len(state_map)+len(employers_id))
    feat[0] = 1
    feat[int(datum['YEAR']) - 2010] = 1
    feat[state_map[datum['STATES']]+6] = 1
    feat[employers_id[datum['EMPLOYER_NAME']]+len(state_map)+6] = 1
    return feat

emp_list = data.EMPLOYER_NAME.unique().tolist()
employers_id = dict(zip(emp_list, range(len(emp_list))))

X_train = [feature(d[1]) for d in data.loc[:500000,:].iterrows() if d[1]['STATES'] != 'NA']
y_train = [d[1]['CASE_STATUS'] for d in data.loc[:500000,:].iterrows() if d[1]['STATES'] != 'NA']

In [8]:
X_validation =  [feature(d[1]) for d in data.loc[500000:1000000,:].iterrows() if d[1]['STATES'] != 'NA']
y_validation =  [d[1]['CASE_STATUS'] for d in data.loc[500000:1000000,:].iterrows() if d[1]['STATES'] != 'NA']

In [9]:
X_test =  [feature(d[1]) for d in data.loc[1000000:,:].iterrows() if d[1]['STATES'] != 'NA']
y_test =  [d[1]['CASE_STATUS'] for d in data.loc[1000000:,:].iterrows() if d[1]['STATES'] != 'NA']

In [11]:
logreg = linear_model.LogisticRegression(C=10)
logreg.fit(X_train, y_train)
pred = logreg.predict(X_test)

In [29]:
classifierStats(pred)

Accuracy = 0.983751667148
FPR = 0.995033998014
FNR = 0.00289457774813


In [27]:
def classifierStats(pred):
    tp = 0
    tn = 0
    fp = 0
    fn = 0
    for i in range(len(pred)):
        if y_test[i] == 1:
            if pred[i] == 1:
                tp += 1
            else:
                fn += 1
        else:
            if pred[i] == 1:
                fp += 1
            else:
                tn += 1
    print("Accuracy = " + str(1.0*(tp + tn)/(tn+fp+fn+tp)))
    print("FPR = " + str(1.0*fp/(tn+fp)))
    print("FNR = " + str(1.0*fn/(tp+fn)))

In [32]:
#Baseline 1 : ZeroR
y_consolidated = y_train + y_validation 

from collections import Counter

def Most_Common(y_consolidated):
    data = Counter(y_consolidated)
    return data.most_common(1)[0][0]

pred_baselise1 = [Most_Common(y_consolidated)] * len(y_test)

#Accuracy & FNR & FPR
classifierStats(pred_baselise1)

Accuracy = 0.986540444991
FPR = 1.0
FNR = 0.0


In [39]:
import random
#Baseline 2 : Biased Coin Toss
sum = numpy.sum(y_consolidated)
sum = sum*1.0/len(y_consolidated)
#Accuracy
pred_baselise2 = [ (random.random() < sum) for x in range(len(y_test)) ]
#FNR & FPR
classifierStats(pred_baselise2)

Accuracy = 0.979645623665
FPR = 0.993047597219
FNR = 0.00708374158311
